# Tuple

## Architecture

**Important:**
- t_xmin - txid of the transaction that inserted this tuple.
- t_xmax - txid of the transaction that deleted or updated this tuple. If this tuple has not been deleted or updated, t_xmax is set to 0, which means INVALID.
- t_cid - the number of SQL commands that were executed before this command was executed within the current transaction, starting from 0.
- t_ctid - tuple identifier (tid) that points to itself or a new tuple (if there is a new version of the tuple), structure: (page_index, tuple_index_in_page).

**Not In Scope:**
- t_infomask, t_infomask2 - contains information about the tuple that reduce work for processes, in our context hint bits about xmin and xmax are stored here too.
- t_hoff - offset to user data in tuple.

<img src="./helpers/tuple-structure.png" alt="drawing" width="900"/>

## Hands On

Operations:
- INSERT
- UPDATE
- DELETE

In [ ]:
psql -c "CREATE EXTENSION pageinspect;"
psql -c "DROP TABLE IF EXISTS tbl; CREATE TABLE tbl (data text);"
psql -c "INSERT INTO tbl VALUES ('A'), ('B'), ('C');"
psql -c "UPDATE tbl SET data = 'D' WHERE data = 'A';"
psql -c "DELETE FROM tbl WHERE data = 'B';"

psql << EOM
    SELECT 
        lp as tuple_num
        ,t_xmin
        ,t_xmax
        ,t_field3 as t_cid
        ,t_ctid 
    FROM heap_page_items(get_raw_page('tbl', 0));
EOM
psql -c "DROP TABLE IF EXISTS tbl;"

## Tuple VS Row

There is sometimes a distinction between a tuple and a row:
- Tuple - a single data object that represent a single version of a row.
- Row - logical table row, can consist of several tuples.